### Projeto de análise de dados sobre os registro de batalhas dos 1000 melhores jogadores em desafio 1v1 no clash royale, **Dado retirado em novembro de 2018**

In [ ]:
# IMPORTANDO O PANDAS
import pandas as pd

In [ ]:
# IMPORTANDO A BASE DE DADOS
df_clash = pd.read_excel('../base-dados/Clash Royale Top Battles v1.xlsx')

In [ ]:
# FORMATANDO E TRATANDO O BÁSICO DO DATASET
df_clash.info()
del df_clash['Unnamed: 0']

### 📍 Extraindo o as cartas mais utilizadas nos decks importados

In [ ]:
# CRIANDO DICIONÁRIO COM OS DADOS DE QUANTIDADE DE APARIÇÕES DAS CARTAS, PASSANDO POR TODAS AS POSIÇÕES POSSÍVEIS DE LOCAL NO DECK, DE 1 A 8
def contar_cartas_utlizada_posi_deck():
    dic_cartas_por_posi = {}
    for x in range(1, 9):
        dic_intermediaria = df_clash[f'p1_{x}'].value_counts().sort_values(ascending=False).to_dict()
        dic_cartas_por_posi.update({f'Posicao_{x}':dic_intermediaria})
    
    
    return dic_cartas_por_posi
        
dic_cartas_por_posi = contar_cartas_utlizada_posi_deck()

In [ ]:
# SOMANDO A QUANTIDADE DE APARIÇÕES NOS DECKS, CONTANDO TODAS AS POSIÇÕES POSSÍVEIS
def somar_total_cartas(valor_dic_cartas):
    # PRIMEIRO FOR COM O DICIONÁRIO INTEIRO, COM TODAS AS POSIÇÕES
    total_cartas = {}
    for i, x in valor_dic_cartas.items():
        # SEGUNDO FOR DENTRO DO DICIONÁRIO COM A QUANTIDADE DE CARTA INDIVIDUAL
        for carta, qtd in x.items():
            if carta not in total_cartas:
                total_cartas.update({carta: qtd})
            else:
               total_cartas[carta] += qtd
               
    return total_cartas

total_cartas = somar_total_cartas(dic_cartas_por_posi)
seris_cartas = pd.Series(total_cartas)

# DATAFRAME TEMPORÁRIO COM O NÚMERO DE VEZES QUE UMA CARTA FOI UTILIZADA
df_carta_mais_usada = pd.DataFrame(seris_cartas,columns=['Quantidade']).sort_values(by='Quantidade',ascending=False)
df_carta_mais_usada

### Verificando quais cartas mais aparecem nos decks vencedores

In [ ]:
# CRIAR UM CÓPIA
df_copia_teste = df_clash.copy()

# VERIFICAR QUAL DECK FOI O VENCEDOR EM CADA PARTIDA
def contar_numero_vitorias(valor):
    if valor['p1_crowns'] > valor['p2_crowns']:
        return 1
    else:
        return 0

# 1 = DECK 1
# 0 = DECK 2
    
mask_vencedores = df_copia_teste[['p1_crowns', 'p2_crowns']].apply(contar_numero_vitorias, axis=1)
# CRIAR A NOVA COLUNA COM QUAL DECK VENCEU TAL PARTIDA
df_copia_teste['Deck Vencedor'] = mask_vencedores

In [ ]:
df_copia_teste.shape

In [ ]:
# VERIFICAR AS APARIÇÕES DAS CARTAS NOS DECKS VENCEDORES
lista_cartas_vitoriosas = {}
def verificar_cartas_decks_vitoriosos(valor):
    # VERIFICAR SE É O P1 OU O P2
    if valor['Deck Vencedor'] == 1:
        p = 1
        
    else:
        p = 2 
    
    # CRIA A FILTRAGEM AUTOMATICAMENTE
    colunas = [f'p{p}_{i}' for i in range(1,9)]
    # RETORNA UM DICIONARIO COM A LISTA DE CARTAS DO DECK
    return {'carta': valor[colunas].tolist()}

# CHAMA A FUNÇÃO PARA TODAS AS LINHAS DO DATAFRAME
resultado = df_copia_teste.apply(verificar_cartas_decks_vitoriosos, axis=1)

# CRIA O DICIONÁRIO FINAL COM TODA A LISTA DE DECK COMPLETA DOS VENCEDORES
for i, x in enumerate(resultado):
    lista_cartas_vitoriosas.update({f'Deck_{i}':x['carta']})
print(lista_cartas_vitoriosas)

In [ ]:
# PASSAR O DICIONÁRIO PARA UM DATAFRAME SIMPLES PARA FAZER UM COUNT
df_com_cartas_vitoriosas = pd.DataFrame({
    'POSI NO DECK: 1': [0],
    'POSI NO DECK: 2': [0],
    'POSI NO DECK: 3': [0],
    'POSI NO DECK: 4': [0],
    'POSI NO DECK: 5': [0],
    'POSI NO DECK: 6': [0],
    'POSI NO DECK: 7': [0],
    'POSI NO DECK: 8': [0]
})

# PASSO PARA O DATA FRAME, COM O DECK SENDO UMA LINHA E CADA CARTA UMA CÉLULA DE UMA COLUNA
for x in range(1, df_clash.shape[0]):
    df_com_cartas_vitoriosas.loc[len(df_com_cartas_vitoriosas)] = lista_cartas_vitoriosas[f'Deck_{x}']
df_com_cartas_vitoriosas.drop(index=0, inplace=True)

In [ ]:

def contar_aparicoes(valor):
    dic_vic = {}
    for x in range(1, 9):
        valor_cartas = valor[f'POSI NO DECK: {x}'].value_counts().to_dict()
        for i, x in valor_cartas.items():
            if i not in dic_vic:
                dic_vic.update({i:x})
            else:
                dic_vic[i] += x
    return dic_vic


dic_vic = contar_aparicoes(df_com_cartas_vitoriosas)
dic_vic = pd.Series(dic_vic)

dic_vic = pd.DataFrame(dic_vic, columns=['Carta Mais Vitoriosa'])
dic_vic.sort_values(by='Carta Mais Vitoriosa', ascending=False)